Steps:
1. Unzip the dm6.fa.zip file. It contains the genome of DM in FASTA format.
2. Unzip the genes.txt.zip file. It contains locations of exons and intros of the DM genes.
3. Install Python and Jupyter notebook.
4. Create a new notebook.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get install -y bedtools
import pandas as pd
!pip install biopython
from Bio import SeqIO
import numpy as np
from itertools import product


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  bedtools
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 563 kB of archives.
After this operation, 1,548 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 bedtools amd64 2.30.0+dfsg-2ubuntu0.1 [563 kB]
Fetched 563 kB in 1s (929 kB/s)
Selecting previously unselected package bedtools.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../bedtools_2.30.0+dfsg-2ubuntu0.1_amd64.deb ...
Unpacking bedtools (2.30.0+dfsg-2ubuntu0.1) ...
Setting up bedtools (2.30.0+dfsg-2ubuntu0.1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.7 MB/s eta 0:00:00


a. Extract the locations of the exons and the introns. Save exon locations to a file in
BED format. Save intron locations to a separate file in BED format. Make sure
that there are no duplicated entries in each file and there are no intersecting
regions between introns and exons. 


https://bedtools.readthedocs.io/en/latest/content/example-usage.html#

In [27]:
genes_file = "/content/drive/MyDrive/mlinmb/1/genes.txt"
columns = ["bin", "name", "chrom", "strand", "txStart", "txEnd", "cdsStart", "cdsEnd", "exonCount", "exonStarts", "exonEnds", "score", "name2", "cdsStartStat", "cdsEndStat", "exonFrames"]
df = pd.read_csv(genes_file, sep="\t", header=None, names=columns, skiprows=1)

exons = []
for i, row in df.iterrows():
    exon_starts = row["exonStarts"].strip(",").split(",")
    exon_ends = row["exonEnds"].strip(",").split(",")

    # Create file with chrom name, start, and end positions
    for j in range(len(exon_starts)):
        start = int(exon_starts[j])
        end = int(exon_ends[j])
        chrom = row["chrom"]
        exons.append((chrom, start, end))

introns = []
for i, row in df.iterrows():
    exon_starts = row["exonStarts"].strip(",").split(",")
    exon_ends = row["exonEnds"].strip(",").split(",")

    for j in range(len(exon_starts) - 1):  # Iterujemy do przedostatniego eksonu
            intron_start = int(exon_ends[j])
            intron_end = int(exon_starts[j + 1])
            chrom = row["chrom"]

            if intron_start < intron_end:  # Sprawdzamy poprawność zakresu
                introns.append((chrom, intron_start, intron_end))
print(f"Number of introns: {len(introns)}")
print(f"Number of exons: {len(exons)}")

# remove duplicates and save to .bed
exons = list(set(exons))
introns = list(set(introns))

with open("exons.bed", "w") as f:
    for exon in exons:
        f.write(f"{exon[0]}\t{exon[1]}\t{exon[2]}\n")

with open("introns.bed", "w") as f:
    for intron in introns:
        f.write(f"{intron[0]}\t{intron[1]}\t{intron[2]}\n")



Number of introns: 154003
Number of exons: 188467


In [29]:
# - v returns regions in a that do not overlap with regions in b
!bedtools intersect -v -a introns.bed -b exons.bed > introns_filtered.bed
!bedtools intersect -v -a exons.bed -b introns.bed > exons_filtered.bed


b. Extract the sequences of exons and save them to a file in FASTA format. 


https://biopython.org/wiki/SeqIO

In [30]:
genome_file = "/content/drive/MyDrive/mlinmb/1/dm6.fa"
genome = SeqIO.to_dict(SeqIO.parse(genome_file, "fasta")) # create dictionary

exons_file = "exons.bed"
exons = []
# add tuple to the list of exons
with open(exons_file, "r") as bed:
    for line in bed:
        chrom, start, end = line.strip().split()[:3]
        exons.append((chrom, int(start), int(end)))
# extract sequence for the particular exon and save to fasta
output_fasta = "exons_sequences.fasta"
with open(output_fasta, "w") as output:
    for i, (chrom, start, end) in enumerate(exons):
        seq = genome[chrom].seq[start:end]
        output.write(f">exon_{i + 1}_{chrom}_{start}_{end}\n{seq}\n")

c. Extract the sequences of introns and save them to a file in FASTA format.

In [31]:
genome_file = "/content/drive/MyDrive/mlinmb/1/dm6.fa"
genome = SeqIO.to_dict(SeqIO.parse(genome_file, "fasta"))

introns_file = "introns.bed"
introns = []
# the same for introns
with open(introns_file, "r") as bed:
    for line in bed:
        chrom, start, end = line.strip().split()[:3]
        introns.append((chrom, int(start), int(end)))

output_fasta = "introns_sequences.fasta"
with open(output_fasta, "w") as output:
    for i, (chrom, start, end) in enumerate(introns):
        seq = genome[chrom].seq[start:end]
        output.write(f">intron_{i + 1}_{chrom}_{start}_{end}\n{seq}\n")

5. Write a Python class that generates a k-mer (k is a parameter that may take a value
between 1 and 6) histogram from a DNA sequence. The order of k-mer is the same across all sequences. For this reason, we do not need to record the k-mers
themselves—only their count. Additionally, all histograms of the same k have the same
size, i.e. 4k. A histogram is available as a numpy array.

In [32]:
class KmerHistogram:
    def __init__(self, k):
        if k < 1 or k > 6:
            raise ValueError("k must be between 1 and 6")
        self.k = k

        # generate a list of all possible kmers
        self.kmers = []
        for p in product('ACGT', repeat=k):
            kmer = ''.join(p)
            self.kmers.append(kmer)

        # assign kmers to indexes in dictionary
        self.kmer_to_index = {}
        for i in range(len(self.kmers)):
            kmer = self.kmers[i]
            self.kmer_to_index[kmer] = i

    def generate_histogram(self, sequence):
        # fill the histogram with zeros
        histogram = np.zeros(len(self.kmers), dtype=int)

        # count kmers
        for i in range(len(sequence) - self.k + 1):
            kmer = sequence[i:i + self.k]
            if kmer in self.kmer_to_index:
                index = self.kmer_to_index[kmer]
                histogram[index] += 1

        return histogram


6. Filter the exons and the introns. Keep sequences that are at least 50 nucleotides and at most 1000 nucleotides.

In [33]:
def filter_sequences(sequences):
    filtered_sequences = []

    for sequence in sequences:
        length = len(sequence)
        if length >= 50 and length <= 1000:
            filtered_sequences.append(sequence)

    return filtered_sequences

# get exon and intron sequences from fasta file
exon_sequences = []
for record in SeqIO.parse("exons_sequences.fasta", "fasta"):
    exon_sequences.append(str(record.seq))

intron_sequences = []
for record in SeqIO.parse("introns_sequences.fasta", "fasta"):
    intron_sequences.append(str(record.seq))


filtered_exons = filter_sequences(exon_sequences)
filtered_introns = filter_sequences(intron_sequences)

print(f"Number of filtered exons: {len(filtered_exons)}")
print(f"Number of filtered introns: {len(filtered_introns)}")


Number of filtered exons: 69286
Number of filtered introns: 46722


7. Convert each sequence to a histogram and collect all histograms (for exons and introns)
into a matrix. This matrix is the features.

In [34]:
def histograms_to_matrix(sequences, kmer_hist):
    histograms = []

    for sequence in sequences:
        histogram = kmer_hist.generate_histogram(sequence)
        histograms.append(histogram)

    features = np.array(histograms)  # list of histograms to matrix
    return features


kmer_hist = KmerHistogram(k=4)

# features matrix
exon_features = histograms_to_matrix(filtered_exons, kmer_hist)
intron_features = histograms_to_matrix(filtered_introns, kmer_hist)

print("Features matrix for exons:")
print(exon_features)

print("Features matrix for introns:")
print(intron_features)

Features matrix for exons:
[[11  8  3 ...  1  2  1]
 [ 7  7  1 ...  0  1  1]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 [ 0  2  1 ...  0  1  0]]
Features matrix for introns:
[[11  7  4 ...  8 10 20]
 [ 0  0  0 ...  0  0  0]
 [ 7  2  2 ...  5  1  5]
 ...
 [ 0  2  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  2  1 ...  1  1  0]]


8. Make another numpy array that has a row entry for each sequence. The value stored in
this entry is 1 if the corresponding sequence in an exon and 0 if the corresponding sequence is an intron. This array is the labels.

In [35]:
def create_labels(exon_count, intron_count):
    # fill array with zeros
    labels = np.zeros(exon_count + intron_count, dtype=int)
    # set 1 for exons
    for i in range(exon_count):
        labels[i] = 1

    return labels

exon_count = len(filtered_exons)
intron_count = len(filtered_introns)

# label matrix
labels = create_labels(exon_count, intron_count)

print("Labels:")
print(labels)


Labels:
[1 1 1 ... 0 0 0]


In [36]:
print(f"Number of labels for exons: {np.sum(labels == 1)}")
print(f"Number of labels for introns: {np.sum(labels == 0)}")


Number of labels for exons: 69286
Number of labels for introns: 46722


9. Make sure using an assert statement that number of rows in the features and the labels
are the same.

In [37]:
def assert_number_of_rows_and_labels(features, labels):
    assert features.shape[0] == labels.shape[0], 'Not the same number'

#
features = np.vstack([exon_features, intron_features])

assert_number_of_rows_and_labels(features, labels)

print("Assert succesful")

print(f"Features Shape: {features.shape}")
print(f"Labels Shape: {labels.shape}")

Assert succesful
Features Shape: (116008, 256)
Labels Shape: (116008,)


10. Save the feature matrix and the label array to the disk.

In [38]:
def save_features_and_labels(features, labels, features_filename, labels_filename):
    np.save(features_filename, features)
    np.save(labels_filename, labels)

features_filename = "/content/drive/MyDrive/mlinmb/1/features.npy"
labels_filename = "/content/drive/MyDrive/mlinmb/1/labels.npy"

save_features_and_labels(features, labels, features_filename, labels_filename)

print(f"Features saved as {features_filename}")
print(f"Labels saved as {labels_filename}")


Features saved as /content/drive/MyDrive/mlinmb/1/features.npy
Labels saved as /content/drive/MyDrive/mlinmb/1/labels.npy
